# Découverte du jeu de données


## Import

In [1]:
from sklearn.decomposition import PCA
import pandas as pd
import os

## Partie importante

In [ ]:
train_path = os.path.abspath(os.path.dirname(__name__)) + '\\petfinder-adoption-prediction\\train\\train.csv'

# Lecture du fichier des données d'entrainement
df = pd.read_csv(train_path, sep=',')

# On récupère les noms de colonnes
columns_names = list(df.columns.values)
print(columns_names)

In [12]:
# On affiche les données
df.drop(columns=['Name', 'Description', 'RescuerID'])
print(df)

       Type                                 Name  Age  Breed1  Breed2  Gender  \
0         2                               Nibble    3     299       0       1   
1         2                          No Name Yet    1     265       0       1   
2         1                               Brisco    1     307       0       1   
3         1                                 Miko    4     307       0       2   
4         1                               Hunter    1     307       0       1   
5         2                                  NaN    3     266       0       2   
6         2                                BULAT   12     264     264       1   
7         1              Siu Pak & Her 6 Puppies    0     307       0       2   
8         2                                  NaN    2     265       0       2   
9         2                                Kitty   12     265       0       2   
10        1                                 Bear    2     307       0       1   
11        2                 

In [ ]:
"""
    À la simple vision des noms de colonnes on peut très vite s'orienter vers un calcul de corrélations avec la rapidité d'adoption :
        - Vaccinated
        - Dewormed
"""
correlation = df.corr(method='pearson')
print(correlation)

In [ ]:
# Pour seuiller les valeurs : on utilise la fonction where comme pour une bdd
cor_seuil = 0.5
correlated = correlation.where((correlation > cor_seuil) | (correlation < -cor_seuil))
print(correlated)
# TODO : Récupérer les valeurs d'index et de colonnes pour des variables corrélées